<a href="https://colab.research.google.com/github/J-Pitts/J-Pitts.github.io/blob/master/AutoImportPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Auto Import PDF, Then Delete

Imports

In [ ]:
import time                                                         # for waiting for pages to load
from pdfminer.high_level import extract_text                        # pdf reader for complete pdfs without fillable fields
import re                                                           # regular expressions module
import traceback
from selenium import webdriver                                      # main webdriver
from selenium.webdriver.chrome.service import Service               # main webdriver Chrome Service
from selenium.webdriver.common.by import By                         # main webdriver Searchable X Path Service
from selenium.webdriver.support.ui import WebDriverWait             # main webdriver Waiting until clickable function
from selenium.webdriver.support import expected_conditions as ec    # main webdriver Error resolve helper
from selenium.webdriver.common.action_chains import ActionChains    # main webdriver being able to press enter/esc/etc.
from webdriver_manager.chrome import ChromeDriverManager            # main webdriver Chrome
from selenium.webdriver.common.keys import Keys                     # main webdriver sending text to fields
import os                                                           # to change directory of file

Pull from PDF using pdfminer, indexed after field name, split if needed

In [ ]:

#--------------------------------> Path to directory of file folder for pdfs <--------------------------
path = r'...\\Applications\\'


#---------------------------> Create main function for pdf read and web import <------------------------
def application_import(app_pdf):


#-----------------------> Read pdf and extract text using pdfminer module extract_text <----------------
    pdf_text_string = extract_text(app_pdf)


#--------------------------------> Split extracted text from pdf by lines <-----------------------------
    pdf_text_split = pdf_text_string.split("\n")


#---------------------------------------> Variables <---------------------------------------------------

    first_name = pdf_text_split[pdf_text_split.index('Name')+1].split(' ')[0]
        #indexed +1 after Name, split name by spaces, pulled first word

    last_name = pdf_text_split[pdf_text_split.index('Name')+1].split(' ')[-1]
        #indexed +1 after Name, split name by spaces, pulled last word


    date_of_birth = []
        # set date_of_birth as empty list

    if len(pdf_text_split[pdf_text_split.index('Date of Birth')+1].split(' ')) == 3:
        for group in pdf_text_split[pdf_text_split.index('Date of Birth')+1].split(' '):
            if len(group) < 3:
                date_of_birth.append(str(group))
            if len(group) > 3:
                date_of_birth.append(str(group))

    elif len(pdf_text_split[pdf_text_split.index('Date of Birth')+1].split(' ')) < 6:
        for group in pdf_text_split[pdf_text_split.index('Date of Birth')+1].split('/'):
        #split date of birth by /, run through each group
            if len(group) >= 2:
                date_of_birth.append(group)
            #if group is 2 or more, add to list
            if len(group) < 2:
                date_of_birth.append(str('0')+str(group))
            #if group is less than 2, add 0 to string before adding to list
            #print(group)
    date_of_birth = ''.join(date_of_birth)
        #combine list of the 3 strings into the final date of birth string without /


    email = pdf_text_split[pdf_text_split.index('Email')+1]
        #indexed +1 after Email

    drivers_license_number = pdf_text_split[pdf_text_split.index('Drivers License Number')+1]
        #indexed +1 after Drivers license number

    ssn = pdf_text_split[pdf_text_split.index('Social Security Number')+1]
        #indexed +1 after Social Security Number

    phone_number = re.sub("[()]","", pdf_text_split[pdf_text_split.index('Phone')+1])
        #indexed +1 after phone number, removed () from phone number for clean entry

    street_address = pdf_text_split[pdf_text_split.index('Address')+1].split(',')[0]
        #indexed +1 after Address, split address by commas, pulled first block for street address

    city = pdf_text_split[pdf_text_split.index('Address')+1].split(',')[-3]
        #indexed +1 after Address, split address by commas, pulled 3rd from last for city

    state = pdf_text_split[pdf_text_split.index('Address')+1].split(' ')[-2]
        #indexed +1 after Address, split address by commas, pulled 2nd from last for state

    zip_code = pdf_text_split[pdf_text_split.index('Address')+1].split(' ')[-1]
        #indexed +1 after Address, split address by commas, pulled last for Zip Code

    employer = pdf_text_split[pdf_text_split.index('Employer')+1]
        #indexed +1 after Employer

    job_title = pdf_text_split[pdf_text_split.index('Job Title')+1]
        #indexed +1 after Job Title

    income = pdf_text_split[pdf_text_split.index('Net Monthly Pay (After Taxes)')+1]
        #indexed +1 after Net Pay

    dealership = pdf_text_split[pdf_text_split.index('Dealership Name (Place Vehicle is Located)')+1].split(' ')[0:1]
        #indexed +1 after Dealership name, split by spaces, only pull first 2


    #Check for Coapp
    if len(pdf_text_split[pdf_text_split.index('Co-Applicant Name (First & Last)')+1]) == 0: coapp = None
    else : coapp = True

    if coapp == True:
        co_applicant_first_name = pdf_text_split[pdf_text_split.index('Co-Applicant Name (First & Last)',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1].split(' ')[0]
        co_applicant_last_name = pdf_text_split[pdf_text_split.index('Co-Applicant Name (First & Last)',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1].split(' ')[-1]

        co_applicant_date_of_birth = []
        for group in pdf_text_split[pdf_text_split.index('Date of Birth',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1].split('/'):
            if len(group) >= 2:
                co_applicant_date_of_birth.append(group)
            if len(group) < 2:
                co_applicant_date_of_birth.append(str('0')+str(group))

        co_applicant_date_of_birth = ''.join(co_applicant_date_of_birth)

        co_applicant_ssn = pdf_text_split[pdf_text_split.index('Social Security Number',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1]
        co_applicant_dl_number = pdf_text_split[pdf_text_split.index('Driver License Number',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1]
        co_applicant_phone_number = re.sub("[()]","", pdf_text_split[pdf_text_split.index('Phone',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1])
        co_applicant_email = pdf_text_split[pdf_text_split.index('Email',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1]
        co_applicant_street_address = pdf_text_split[pdf_text_split.index('Address',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1].split(',')[0]
        co_applicant_city = pdf_text_split[pdf_text_split.index('Address',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1].split(',')[-3]
        co_applicant_state = pdf_text_split[pdf_text_split.index('Address',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1].split(' ')[-2]
        co_applicant_zip_code = pdf_text_split[pdf_text_split.index('Address',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1].split(' ')[-1]
        co_applicant_employer = pdf_text_split[pdf_text_split.index('Employer',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1]
        co_applicant_job_title = pdf_text_split[pdf_text_split.index('Job Title',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1]
        co_applicant_income = pdf_text_split[pdf_text_split.index('Net Monthly Pay (After Taxes)',pdf_text_split.index('Dealership Name (Place Vehicle is Located)'))+1]


Now that I have my data, time to navigate to company portal

In [ ]:

#----------------------------------> URLs For Driver <---------------------------------------------------
    url = (".../auth/login/")
    application_url = (".../applications/newapp/")

#-----------------------> Start chrome webdriver <-------------------------------------------------------
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


#----------------------------> Start browser and go to Co. URL <-----------------------------------------
    driver.get(url)


#----------------------------> Pass Login Info To Co. Page & Login <-------------------------------------
    company_id = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"/html/body/div[2]/div/div[1]/div/div/div/div/form/div[1]/input")))
    company_id.send_keys(companyid)

    username = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"/html/body/div[2]/div/div[1]/div/div/div/div/form/div[2]/input")))
    username.send_keys(username)

    password = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"/html/body/div[2]/div/div[1]/div/div/div/div/form/div[3]/input")))
    password.send_keys(password)

    login = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"/html/body/div[2]/div/div[1]/div/div/div/div/form/button")))
    login.click()

    time.sleep(3)


#------------------> Once logged into Co., click new application url using side menu <------------

    WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='side-menu']/li[1]/ul/li[2]/a"))).click()

    time.sleep(3)

    if coapp == True :
        #click add coapplicant
        WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='calx_formula_container']/div[1]/div[2]/div[2]/button[2]"))).click()

    first_name_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_input_primaryApplicant.firstName_']")))
    first_name_field.send_keys(first_name)

    phone_number_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_input_primaryApplicant.mobilePhone_']")))
    phone_number_field.send_keys(phone_number)

    last_name_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_input_primaryApplicant.lastName_']")))
    last_name_field.send_keys(last_name)

    ssn_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_input_primaryApplicant.SSN_']")))
    ssn_field.send_keys(ssn)

    date_of_birth_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_datepicker_primaryApplicant.dateOfBirth_']")))
    date_of_birth_field.send_keys(date_of_birth)

    us_citizen_clickcircle = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='Applicant Info']/div[2]/div/div[1]/formly-field/div/div[3]/div[2]/formly-field/div/div[3]/label[1]/div")))
    us_citizen_clickcircle.click()

    drivers_license_number_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_input_primaryApplicant.licenseNumber_']")))
    drivers_license_number_field.send_keys(drivers_license_number)

    state_dropdown = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_ui-select_primaryApplicant.driverState_']/div/span")))
    state_dropdown.click()
    state_choice = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_ui-select_primaryApplicant.driverState_']/input[1]")))
    state_choice.send_keys(state)
    state_choice.send_keys(Keys.ENTER)

    email_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_input_primaryApplicant.email_']")))
    email_field.send_keys(email)

    address_line_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='primaryApplicant.addresses_addressLine1_addressLine1primaryApplicant.addresses-0']")))
    address_line_field.send_keys(street_address)

    zip_code_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='primaryApplicant.addresses_postalCode_postalCodeprimaryApplicant.addresses-0']")))
    zip_code_field.send_keys(zip_code)

    employer_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='primaryApplicant.employments_employerName_employerNameprimaryApplicant.employments-0']")))
    employer_field.send_keys(employer)

    job_title_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='primaryApplicant.employments_title_titleprimaryApplicant.employments-0']")))
    job_title_field.send_keys(job_title)

    income_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='primaryApplicant.employments_grossIncome_formly_2_dl4Number_grossIncome_primaryApplicant.employments-0']")))
    income_field.send_keys(income)

    # if coapp
    if coapp == True :
        #click add coapplicant
        co_applicant_first_name_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_input_coapplicant.firstName_']")))
        co_applicant_first_name_field.send_keys(co_applicant_first_name)

        co_applicant_last_name_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_input_coapplicant.lastName_']")))
        co_applicant_last_name_field.send_keys(co_applicant_last_name)

        co_applicant_date_of_birth_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_datepicker_coapplicant.dateOfBirth_']")))
        co_applicant_date_of_birth_field.send_keys(co_applicant_date_of_birth)

        co_applicant_phone_number_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_input_coapplicant.mobilePhone_']")))
        co_applicant_phone_number_field.send_keys(co_applicant_phone_number)

        co_applicant_ssn_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_input_coapplicant.SSN_']")))
        co_applicant_ssn_field.send_keys(co_applicant_ssn)

        #click citizen
        WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='Applicant Info']/div[2]/div/div[2]/formly-field/div/div[2]/div/formly-field/div/div[2]/div/formly-field/div/div[3]/label[1]/div"))).click()

        co_applicant_email_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_input_coapplicant.email_']")))
        co_applicant_email_field.send_keys(co_applicant_email)

        co_applicant_dl_number_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_input_coapplicant.licenseNumber_']")))
        co_applicant_dl_number_field.send_keys(co_applicant_dl_number)

        co_applicant_state_dropdown = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_ui-select_coapplicant.driverState_']/div/span")))
        co_applicant_state_dropdown.click()
        co_applicant_state_choice = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='formly_2_ui-select_coapplicant.driverState_']/input[1]")))
        co_applicant_state_choice.send_keys('Florida')
        co_applicant_state_choice.send_keys(Keys.ENTER)

        #click add address
        WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"/html/body/div[2]/div[3]/div/div[3]/form/div[1]/div[4]/div/div[2]/div[2]/div[2]/div/ng-form/div[1]/div/div/div[2]/formly-field/div/div[2]/div/div/formly-field/div/button"))).click()
        co_applicant_address_line_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='coapplicant.addresses_addressLine1_addressLine1coapplicant.addresses-0']")))
        co_applicant_address_line_field.send_keys(co_applicant_street_address)

        co_applicant_zip_code_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='coapplicant.addresses_postalCode_postalCodecoapplicant.addresses-0']")))
        co_applicant_zip_code_field.send_keys(co_applicant_zip_code)

        #Add employment button
        WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"/html/body/div[2]/div[3]/div/div[3]/form/div[1]/div[4]/div/div[2]/div[2]/div[3]/div/ng-form/div[1]/div/div/div/formly-field/div/div[2]/div/div/formly-field/div/button"))).click()
        co_applicant_employer_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='coapplicant.employments_employerName_employerNamecoapplicant.employments-0']")))
        co_applicant_employer_field.send_keys(co_applicant_employer)

        co_applicant_job_title_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='coapplicant.employments_title_titlecoapplicant.employments-0']")))
        co_applicant_job_title_field.send_keys(co_applicant_job_title)

        co_applicant_income_field = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='coapplicant.employments_grossIncome_formly_69_dl4Number_grossIncome_coapplicant.employments-0']")))
        co_applicant_income_field.send_keys(co_applicant_income)

    #Input dummy bike valuation, requirement of software
    value_source_dropdown = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='loanInformation.collaterals_valueSource_valueSourceMotorcycle-0']/div/span")))
    value_source_dropdown.click()
    value_source_choice = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='ui-select-choices-row-7-']/a/div")))
    value_source_choice.click()

    time.sleep(2)

    #Input dummy bike, requirement of software
    dummy_bike_mileage = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='loanInformation.collaterals_mileage_formly_2_dl4Number_mileage_Motorcycle-0']")))
    dummy_bike_mileage.send_keys('10')
    dummy_bike_year_dropdown = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='loanInformation.collaterals_year_yearMotorcycle-0']/div/span")))
    dummy_bike_year_dropdown.click()
    dummy_bike_year_choice = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='loanInformation.collaterals_year_yearMotorcycle-0']/input[1]")))
    dummy_bike_year_choice.send_keys('2022')
    dummy_bike_year_choice.send_keys(Keys.ENTER)

    time.sleep(2)

    dummy_bike_make_dropdown = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='loanInformation.collaterals_make_makeMotorcycle-0']/div/span")))
    dummy_bike_make_dropdown.click()
    dummy_bike_make_choice = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='loanInformation.collaterals_make_makeMotorcycle-0']/input[1]")))
    dummy_bike_make_choice.send_keys('HONDA')
    dummy_bike_make_choice.send_keys(Keys.ENTER)

    time.sleep(15)

    dummy_bike_model_dropdown = WebDriverWait(driver,40).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='loanInformation.collaterals_model_modelMotorcycle-0']/div/span")))
    dummy_bike_model_dropdown.click()
    dummy_bike_model_choice = WebDriverWait(driver,60).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='loanInformation.collaterals_model_modelMotorcycle-0']/input[1]")))
    dummy_bike_model_choice.send_keys('CBR300R')
    dummy_bike_model_choice.send_keys(Keys.ENTER)

    dummy_bike_bookout = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='Vehicle']/div[2]/div/div/formly-field/div/div[1]/div[1]/div/div[14]/div/div/button[1]")))
    dummy_bike_bookout.click()


    time.sleep(20)

    #Dealership is dangerous field, no validation. Try with entered, otherwise move for manual review
    try:
        dealership_dropdown = WebDriverWait(driver, 20).until(ec.element_to_be_clickable((By.XPATH, "//*[@id='formly_2_ui-select_internalDealerId_']/div/span"))).click()
        dealership_dropdown2 = WebDriverWait(driver, 20).until(ec.element_to_be_clickable((By.XPATH, "//*[@id='formly_2_ui-select_internalDealerId_']/input[1]")))
        dealership_dropdown2.send_keys(dealership)
        dealership_dropdown2.send_keys(Keys.ENTER)

    except:
        dealership_dropdown = WebDriverWait(driver, 20).until(ec.element_to_be_clickable((By.XPATH, "//*[@id='formly_2_ui-select_internalDealerId_']/div/span"))).click()
        dealership_dropdown2 = WebDriverWait(driver, 20).until(ec.element_to_be_clickable((By.XPATH, "//*[@id='formly_2_ui-select_internalDealerId_']/input[1]")))
        dealership_dropdown2.send_keys('manual_review')
        dealership_dropdown2.send_keys(Keys.ENTER)

    terms_conditions = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='Terms and Conditions']/div[2]/div/div[2]/formly-field/div/div[1]/div/formly-field/div/div[3]/label/div")))
    terms_conditions.click()

    application_submit = WebDriverWait(driver,20).until(ec.element_to_be_clickable((By.XPATH,"//*[@id='calx_formula_container']/div[1]/div[4]/div/div/div[3]/button[2]")))
    application_submit.click()

    #System is slow, wait to ensure submission
    time.sleep(60)





Application is submitted through Co. portal, time to cleanup

In [ ]:
#Continous while loop because I want it to keep checking for new files
while True:
    # Try to find a pdf in the file folder for applications
    try:
        os.chdir(os.path.dirname(path) or '.')
        files = [f for f in os.listdir('.') if os.path.isfile(f)]
        files = filter(lambda f: f.endswith(('.pdf','.PDF')), files)
        filecount = 0
        
        #Count number of pdf files - Check in morning to see if any remaining for previous day
        for file in files:
            print(file)
            filecount = filecount + 1

        print('Number of files in the folder: ', filecount)
        #Get application name with path and file name
        app_pdf = path + file
        
        #Run main functions now that I have pdf name
        application_import(app_pdf)

        #Remove file now that its submitted
        os.remove(app_pdf)
    #If theres no pdfs in folder, say so, wait 3 mins before the while loop restarts
    except Exception as e:
        print(e)
        traceback.print_exc()
        print('No Applications In Folder, Will Check Again In 3 Minutes')
        time.sleep(180)